In [1]:
!wget https://archive.apache.org/dist/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz
!rm spark-3.3.1-bin-hadoop3.tgz   # Tidying up

--2025-03-23 22:05:28--  https://archive.apache.org/dist/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299350810 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.1-bin-hadoop3.tgz’

spark-3.3.1-bin-had 100%[===================>] 285.48M  1.56MB/s    in 3m 11s  

2025-03-23 22:08:40 (1.50 MB/s) - ‘spark-3.3.1-bin-hadoop3.tgz’ saved [299350810/299350810]



In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget https://archive.apache.org/dist/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz
!rm spark-3.3.1-bin-hadoop3.tgz

--2025-03-23 22:08:45--  https://archive.apache.org/dist/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299350810 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.1-bin-hadoop3.tgz’

spark-3.3.1-bin-had 100%[===================>] 285.48M  1.56MB/s    in 3m 11s  

2025-03-23 22:11:56 (1.50 MB/s) - ‘spark-3.3.1-bin-hadoop3.tgz’ saved [299350810/299350810]



In [4]:
# Setting up our environmental variables:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [5]:
!pip install -q findspark
import findspark
findspark.init()

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) #  This will format our output tables a bit nicer when not using the show() method
spark

In [7]:
import multiprocessing
print(multiprocessing.cpu_count())

2


In [8]:
from google.colab import files
uploaded = files.upload()

Saving game.csv.zip to game.csv.zip


In [9]:
!unzip game.csv.zip -d "/content"

Archive:  game.csv.zip
replace /content/game.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/game.csv       


In [8]:
import psutil
import time

def print_memory_usage(label=""):
    process = psutil.Process(os.getpid())
    mem_bytes = process.memory_info().rss
    mem_kb = mem_bytes / 1024
    print(f"{label}Memory Usage: {mem_kb:.2f} KB")


In [9]:
df = spark.read.options(delimiter=",", header=True, inferSchema=True).csv("/content/game.csv")

In [ ]:
df.printSchema()

In [ ]:
df.select("*").show(10)

In [10]:
total_goals = df.selectExpr("SUM(home_goals) as Total_Home_Goals", "SUM(away_goals) as Total_Away_Goals")

In [ ]:
total_goals.show()

In [11]:
total_goals = df.selectExpr(
    "SUM(home_goals) as Total_Home_Goals",
    "SUM(away_goals) as Total_Away_Goals",
    "COUNT(*) as Total_Games"
)

In [12]:
avg_goals = total_goals.selectExpr(
    "Total_Home_Goals / Total_Games as Avg_Home_Goals",
    "Total_Away_Goals / Total_Games as Avg_Away_Goals"
)

In [13]:
print_memory_usage("Before")
total_goals.show()
avg_goals.show()
print_memory_usage("After")

BeforeMemory Usage: 109448.00 KB
+----------------+----------------+-----------+
|Total_Home_Goals|Total_Away_Goals|Total_Games|
+----------------+----------------+-----------+
|           77836|           70707|      26305|
+----------------+----------------+-----------+

+------------------+-----------------+
|    Avg_Home_Goals|   Avg_Away_Goals|
+------------------+-----------------+
|2.9589811822847367|2.687968066907432|
+------------------+-----------------+

AfterMemory Usage: 109456.00 KB


In [14]:
from pyspark.sql.functions import sum, count, col

In [15]:
team_goals = df.groupBy("home_team_id").agg(
    sum("home_goals").alias("Total_Home_Goals"),
    sum("away_goals").alias("Total_Away_Goals"),
    count("*").alias("Total_Games")
)

In [16]:
team_avg_goals = team_goals.selectExpr(
    "home_team_id as Team_ID",
    "Total_Home_Goals",
    "Total_Away_Goals",
    "Total_Games",
    "Total_Home_Goals / Total_Games as Avg_Home_Goals",
    "Total_Away_Goals / Total_Games as Avg_Away_Goals"
)

In [17]:
print_memory_usage("")
team_avg_goals.show(10)
print_memory_usage("")

Memory Usage: 109652.00 KB
+-------+----------------+----------------+-----------+------------------+------------------+
|Team_ID|Total_Home_Goals|Total_Away_Goals|Total_Games|    Avg_Home_Goals|    Avg_Away_Goals|
+-------+----------------+----------------+-----------+------------------+------------------+
|     53|             862|             877|        320|           2.69375|          2.740625|
|     28|            2786|            2261|        896|          3.109375|         2.5234375|
|     26|            2434|            2234|        868| 2.804147465437788|2.5737327188940093|
|     27|            1502|            1466|        527|2.8500948766603416| 2.781783681214421|
|     12|            2418|            2322|        849|2.8480565371024733| 2.734982332155477|
|     22|            2382|            2450|        839| 2.839094159713945|2.9201430274135878|
|      1|            2347|            2105|        843|2.7841043890865955|2.4970344009489915|
|     52|            1411|       

In [ ]:
team_avg_goals = team_avg_goals.orderBy("Team_ID")
team_avg_goals.show(10)

+-------+----------------+----------------+-----------+------------------+------------------+
|Team_ID|Total_Home_Goals|Total_Away_Goals|Total_Games|    Avg_Home_Goals|    Avg_Away_Goals|
+-------+----------------+----------------+-----------+------------------+------------------+
|      1|            2347|            2105|        843|2.7841043890865955|2.4970344009489915|
|      2|            2527|            2491|        870|2.9045977011494255| 2.863218390804598|
|      3|            2570|            2358|        885| 2.903954802259887| 2.664406779661017|
|      4|            2636|            2362|        869|3.0333716915995397| 2.718066743383199|
|      5|            2870|            2472|        896|          3.203125|2.7589285714285716|
|      6|            2773|            2312|        917| 3.023991275899673|2.5212649945474372|
|      7|            2428|            2327|        835|2.9077844311377246|2.7868263473053894|
|      8|            2475|            2272|        869| 2.84

In [ ]:
team_avg_goals = team_avg_goals.orderBy(col("Avg_Home_Goals").desc())
team_avg_goals.show(10)

+-------+----------------+----------------+-----------+------------------+------------------+
|Team_ID|Total_Home_Goals|Total_Away_Goals|Total_Games|    Avg_Home_Goals|    Avg_Away_Goals|
+-------+----------------+----------------+-----------+------------------+------------------+
|     87|              26|              24|          4|               6.5|               6.0|
|     89|              20|              40|          4|               5.0|              10.0|
|     90|              10|               8|          2|               5.0|               4.0|
|     54|             789|             602|        239| 3.301255230125523| 2.518828451882845|
|      5|            2870|            2472|        896|          3.203125|2.7589285714285716|
|     14|            2873|            2491|        904|3.1780973451327434|2.7555309734513274|
|     52|            1411|            1243|        448|3.1495535714285716|2.7745535714285716|
|     15|            2793|            2374|        893| 3.12

In [18]:
overall_avg_home_goals = avg_goals.collect()[0]["Avg_Home_Goals"]
print(f"Overall Avg Home Goals: {overall_avg_home_goals}")

Overall Avg Home Goals: 2.9589811822847367


In [19]:
print_memory_usage("")
filtered_teams = team_avg_goals.filter(col("Avg_Home_Goals") > overall_avg_home_goals)


filtered_teams.show(50)
print_memory_usage("")

Memory Usage: 109904.00 KB
+-------+----------------+----------------+-----------+------------------+------------------+
|Team_ID|Total_Home_Goals|Total_Away_Goals|Total_Games|    Avg_Home_Goals|    Avg_Away_Goals|
+-------+----------------+----------------+-----------+------------------+------------------+
|     28|            2786|            2261|        896|          3.109375|         2.5234375|
|     52|            1411|            1243|        448|3.1495535714285716|2.7745535714285716|
|     16|            2635|            2327|        884| 2.980769230769231|2.6323529411764706|
|      6|            2773|            2312|        917| 3.023991275899673|2.5212649945474372|
|     20|            2523|            2326|        850| 2.968235294117647| 2.736470588235294|
|     54|             789|             602|        239| 3.301255230125523| 2.518828451882845|
|      5|            2870|            2472|        896|          3.203125|2.7589285714285716|
|     15|            2793|       

In [20]:
overall_avg_home_goals = avg_goals.collect()[0]["Avg_Home_Goals"]
overall_avg_away_goals = avg_goals.collect()[0]["Avg_Away_Goals"]

In [21]:
top_teams = team_avg_goals.filter(
    (col("Avg_Home_Goals") > overall_avg_home_goals) &
    (col("Avg_Away_Goals") > overall_avg_away_goals)
)

In [22]:
print_memory_usage("")
top_teams.orderBy("Team_ID").show()
time.sleep(1)
print_memory_usage("")

Memory Usage: 109904.00 KB
+-------+----------------+----------------+-----------+------------------+------------------+
|Team_ID|Total_Home_Goals|Total_Away_Goals|Total_Games|    Avg_Home_Goals|    Avg_Away_Goals|
+-------+----------------+----------------+-----------+------------------+------------------+
|      4|            2636|            2362|        869|3.0333716915995397| 2.718066743383199|
|      5|            2870|            2472|        896|          3.203125|2.7589285714285716|
|      9|            2617|            2329|        856| 3.057242990654206|2.7207943925233646|
|     10|            2615|            2492|        851| 3.072855464159812| 2.928319623971798|
|     14|            2873|            2491|        904|3.1780973451327434|2.7555309734513274|
|     20|            2523|            2326|        850| 2.968235294117647| 2.736470588235294|
|     21|            2638|            2351|        863|3.0567786790266513| 2.724217844727694|
|     52|            1411|       

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving game_teams_stats.csv.zip to game_teams_stats.csv.zip


In [26]:
!unzip game_teams_stats.csv.zip -d "/content"

Archive:  game_teams_stats.csv.zip
replace /content/game_teams_stats.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/game_teams_stats.csv  


In [ ]:
df2 = spark.read.options(delimiter=",", header=True, inferSchema=True).csv("/content/game_teams_stats.csv")

In [ ]:
df2 = df2.withColumnRenamed("team_id", "Team_ID")

In [ ]:
df_combined = team_avg_goals.join(df2, "Team_ID", "inner")

In [ ]:
team_shots = df2.groupBy("team_id").agg(
    sum("shots").alias("Total_Shots"),
    count("*").alias("Total_Games")
)

In [ ]:
team_avg_shots = team_shots.selectExpr(
    "team_id as Team_ID",
    "Total_Shots",
    "Total_Games",
    "Total_Shots / Total_Games as Avg_Shots_Per_Game"
)

In [ ]:
team_avg_shots = team_avg_shots.orderBy("Team_ID")

In [ ]:
team_avg_shots.show()

+-------+-----------+-----------+------------------+
|Team_ID|Total_Shots|Total_Games|Avg_Shots_Per_Game|
+-------+-----------+-----------+------------------+
|      1|    49450.0|       1691|29.243051448846835|
|      2|    51949.0|       1743|29.804360298336203|
|      3|    53456.0|       1768|30.235294117647058|
|      4|    52992.0|       1735| 30.54293948126801|
|      5|    54643.0|       1786| 30.59518477043673|
|      6|    57517.0|       1822|31.568057080131723|
|      7|    49328.0|       1669| 29.55542240862792|
|      8|    51958.0|       1732|29.998845265588916|
|      9|    52188.0|       1709| 30.53715623171445|
|     10|    51796.0|       1709|30.307782328847278|
|     11|    22951.0|        820|  27.9890243902439|
|     12|    53101.0|       1705|31.144281524926686|
|     13|    51270.0|       1682| 30.48156956004756|
|     14|    53598.0|       1804|29.710643015521065|
|     15|    53067.0|       1789|29.662940190050307|
|     16|    54702.0|       1771| 30.887634105

In [33]:
team_avg_shots = team_avg_shots.withColumnRenamed("Team_ID", "Team_ID_Shots")

In [34]:
df_merged = team_avg_goals.join(team_avg_shots, team_avg_goals.Team_ID == team_avg_shots.Team_ID_Shots, "inner") \
                          .drop("Team_ID_Shots")

In [36]:
df_merged = df_merged.orderBy("Team_ID")

In [ ]:
df_merged.show()

+-------+----------------+----------------+-----------+------------------+------------------+-----------+-----------+------------------+
|Team_ID|Total_Home_Goals|Total_Away_Goals|Total_Games|    Avg_Home_Goals|    Avg_Away_Goals|Total_Shots|Total_Games|Avg_Shots_Per_Game|
+-------+----------------+----------------+-----------+------------------+------------------+-----------+-----------+------------------+
|      1|            2347|            2105|        843|2.7841043890865955|2.4970344009489915|    49450.0|       1691|29.243051448846835|
|      2|            2527|            2491|        870|2.9045977011494255| 2.863218390804598|    51949.0|       1743|29.804360298336203|
|      3|            2570|            2358|        885| 2.903954802259887| 2.664406779661017|    53456.0|       1768|30.235294117647058|
|      4|            2636|            2362|        869|3.0333716915995397| 2.718066743383199|    52992.0|       1735| 30.54293948126801|
|      5|            2870|            247

In [37]:
df_final = df_merged.withColumn(
    "Goal_Conversion_Rate",
    ((col("Avg_Home_Goals") + col("Avg_Away_Goals")) / col("Avg_Shots_Per_Game")) * 100
)

In [38]:
df_final = df_final.withColumn("Goal_Conversion_Rate", col("Goal_Conversion_Rate").cast("double"))

In [ ]:
df_final.orderBy(col("Goal_Conversion_Rate").desc()).show()

+-------+----------------+----------------+-----------+------------------+------------------+-----------+-----------+------------------+--------------------+
|Team_ID|Total_Home_Goals|Total_Away_Goals|Total_Games|    Avg_Home_Goals|    Avg_Away_Goals|Total_Shots|Total_Games|Avg_Shots_Per_Game|Goal_Conversion_Rate|
+-------+----------------+----------------+-----------+------------------+------------------+-----------+-----------+------------------+--------------------+
|     89|              20|              40|          4|               5.0|              10.0|       80.0|          4|              20.0|                75.0|
|     87|              26|              24|          4|               6.5|               6.0|      106.0|          6|17.666666666666668|   70.75471698113208|
|     90|              10|               8|          2|               5.0|               4.0|       96.0|          4|              24.0|                37.5|
|     11|            1165|            1283|        4

In [39]:
df_final.select("Team_ID", "Goal_Conversion_Rate") \
        .orderBy(col("Goal_Conversion_Rate").desc()) \
        .show()

+-------+--------------------+
|Team_ID|Goal_Conversion_Rate|
+-------+--------------------+
|     89|                75.0|
|     87|   70.75471698113208|
|     90|                37.5|
|     11|  21.332403816827153|
|     22|   20.25971018506305|
|     14|  19.971389765897353|
|     10|   19.80077269599375|
|     27|  19.705112724529386|
|     15|  19.506206008753416|
|     20|  19.497514854991056|
|      5|   19.48690166823093|
|     52|   19.39251352105704|
|      2|  19.352255959259775|
|     21|   19.33066428925526|
|      7|  19.267566877273673|
|     23|  19.114462939477086|
|      9|  18.921334191481698|
|     24|  18.878394510027267|
|     30|   18.86902529482852|
|     29|   18.83493943174964|
+-------+--------------------+
only showing top 20 rows

